In [16]:
import numpy as np

In [17]:
from braket.aws import AwsDevice, AwsQuantumTask
from braket.circuits import Circuit
from braket.devices import LocalSimulator 
from braket.tracking import Tracker

# AWS local simulator

#device = LocalSimulator()

# AWS online simulator (not free)
#device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")

# DM1 Simulator (adds noise)
device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/dm1")

# Lucy
#device = AwsDevice("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")

In [19]:
from core import TwoBitCircuit_noCode, ThreeBitCircuit_noCode, FourBitCircuit_noCode, FiveBitCircuit_noCode, SixBitCircuit_noCode
from braket.circuits import Circuit, noises, Gate

#Data collection for dependence of Shannon Entropy and Minimum Entropy based upon number of qubits 

shotsnum = 5000
cq = 0
aq = 1

TwoCircuit = TwoBitCircuit_noCode(cq,aq).circuit
ThreeCircuit = ThreeBitCircuit_noCode().circuit
FourCircuit = FourBitCircuit_noCode().circuit
FiveCircuit = FiveBitCircuit_noCode().circuit
SixCircuit = SixBitCircuit_noCode().circuit 
circuits = [TwoCircuit, ThreeCircuit, FourCircuit, FiveCircuit, SixCircuit]

In [20]:
#Introducing error through the AWS Braket SDK. Operates on quantum states through quantum channels essentially applying a projection of the 
#density matrix. Use depolarizing error because it represents information loss due to coupling with environment and represents the 
#Github method of applying superposition of pauli matrices to the states to introduce error. Error is applied over the identity matrix
#after the entangled state to represent the decoherence of the quantum state.
noise1 = noises.Depolarizing(probability=0.05)#arbitrary
noise2 = noises.BitFlip(probability=0.05)#arbitrary
for L in circuits:
    L.apply_gate_noise(noise1)
    L.apply_readout_noise(noise2)


In [21]:
with Tracker() as tracker:
    Two_task = device.run(TwoCircuit, shots=shotsnum)
    Three_task = device.run(ThreeCircuit, shots=shotsnum)
    Four_task = device.run(FourCircuit, shots=shotsnum)
    Five_task = device.run(FiveCircuit, shots=shotsnum)
    Six_task = device.run(SixCircuit, shots=shotsnum)

In [22]:
print(tracker.quantum_tasks_statistics())
print(tracker.simulator_tasks_cost())
print(tracker.qpu_tasks_cost())
print(Two_task.id)
print(Three_task.id)
print(Four_task.id)
print(Five_task.id)
print(Six_task.id)

{'arn:aws:braket:::device/quantum-simulator/amazon/dm1': {'shots': 25000, 'tasks': {'CREATED': 5}}}
0
0
arn:aws:braket:us-east-1:592242689881:quantum-task/58dded9f-c3a0-4ba7-b5d7-7af8961a3c3e
arn:aws:braket:us-east-1:592242689881:quantum-task/bc20a881-3486-4fd3-b11a-4fd567fa0a84
arn:aws:braket:us-east-1:592242689881:quantum-task/9eb60be3-14f7-4659-bd20-60014ca82d49
arn:aws:braket:us-east-1:592242689881:quantum-task/407c6a5a-1057-4b30-9cf7-e000a330feed
arn:aws:braket:us-east-1:592242689881:quantum-task/a808d1d5-7426-43e7-aecf-f770da8522cf


In [23]:
Two_prob = AwsQuantumTask(arn = Two_task.id).result().measurement_probabilities
Three_prob = AwsQuantumTask(arn = Three_task.id).result().measurement_probabilities
Four_prob = AwsQuantumTask(arn = Four_task.id).result().measurement_probabilities
Five_prob = AwsQuantumTask(arn = Five_task.id).result().measurement_probabilities
Six_prob = AwsQuantumTask(arn = Six_task.id).result().measurement_probabilities
print(Two_prob)
print(Three_prob)
print(Four_prob)
print(Five_prob)
print(Six_prob)

{'11': 0.4076, '00': 0.3978, '01': 0.095, '10': 0.0996}
{'001': 0.3828, '100': 0.0474, '010': 0.0586, '101': 0.051, '000': 0.384, '011': 0.0588, '110': 0.0088, '111': 0.0086}
{'1100': 0.0328, '0100': 0.1828, '0000': 0.1868, '0110': 0.0276, '0001': 0.187, '0101': 0.1884, '1011': 0.0048, '1101': 0.0294, '1111': 0.0046, '0111': 0.0288, '0010': 0.028, '1000': 0.0288, '0011': 0.0308, '1110': 0.0032, '1001': 0.0312, '1010': 0.005}
{'01101': 0.0276, '00100': 0.1668, '00101': 0.1604, '10101': 0.0198, '00001': 0.1592, '00000': 0.1664, '11100': 0.0086, '10100': 0.0212, '00010': 0.0228, '10000': 0.0196, '00111': 0.0208, '10001': 0.021, '11001': 0.011, '01100': 0.0232, '00110': 0.02, '01000': 0.0306, '00011': 0.0206, '01110': 0.0038, '10010': 0.002, '01001': 0.03, '11000': 0.0114, '11101': 0.0126, '10110': 0.0024, '01111': 0.0036, '01010': 0.003, '01011': 0.0028, '10011': 0.0024, '11110': 0.0016, '10111': 0.0016, '11111': 0.0012, '11010': 0.001, '11011': 0.001}
{'100000': 0.0234, '111101': 0.0062,

In [24]:
Two_raw = AwsQuantumTask(arn = Two_task.id).result().measurements
Three_raw = AwsQuantumTask(arn = Three_task.id).result().measurements
Four_raw = AwsQuantumTask(arn = Four_task.id).result().measurements
Five_raw = AwsQuantumTask(arn = Five_task.id).result().measurements
Six_raw = AwsQuantumTask(arn = Six_task.id).result().measurements
np.savetxt('braketError_Two_raw.txt',Two_raw,fmt="%i")
np.savetxt('braketError_Three_raw.txt',Three_raw,fmt="%i")
np.savetxt('braketError_Four_raw.txt',Four_raw,fmt="%i")
np.savetxt('braketError_Five_raw.txt',Five_raw,fmt="%i")
np.savetxt('braketError_Six_raw.txt',Six_raw,fmt="%i")